## <span style="color:green">**Version en cours**</span>

# Segmentez des clients d'un site e-commerce
## Notebook 4 : Simulation d'un contrat de maintenance
OpenClassrooms - Parcours Data Scientist - Projet 05  

## Présentation du projet

**Contexte**  


* Olist est une entreprise brésilienne qui propose une solution de vente sur les marketplaces en ligne.  
* Dans un premier temps il est demandé de réaliser quelques requêtes pour le dashboard à partir de la base de données SQLite d'Olist.  
* La mission principale est de fournir aux équipes d'e-commerce d'Olist une **segmentation des clients** qu’elles pourront utiliser au quotidien pour leurs campagnes de communication.

**Démarche globale**  
* Requêtes SQL pour le dashboard (cf Notebook 1)  
* Feature ingineering (cf Notebook 2)
* Tests de modèles de clustering : (cf Notebook 3)  
* **Simulation d'un contrat de maintenance** : c'est l'objet de ce notebook 4  

**Simulation d'un contrat de maintenance**  
* Objectif : déterminer au bout de combien de temps le modèle retenu doit être réentrainé  
* Démarche :
  * Comparer les résultats du dernier clustering réalisé (fin août 2018) avec les prédictions d'un modèle entrainé X jours avant  
  * La comparaison sera mesurée par le score ARI (Adjusted Rand Index) qui mesure la divergence entre clusters  
  * Si les score obtenu est supérieur à 0.80, considérer que les prédictions sont suffisament fiables et on retester avec les prédictions d'un modèle entraîné X jours avant
  * Le délai avant de devoir réentraîner le modèle sera déterminé dès que le score ARI sera descendu sous le seuil de 0.80

## Sommaire  
**Préparation de l'environnement**  
* Environnement virtuel
* Import des modules
* Fonctions

**Préparation des données**  


# 1 Préparation de l'environnement

## 1.1 Environnement virtuel

In [1]:
# Vérification environnement virtuel
envs = !conda env list
print(f"Environnement virtuel : {[e for e in envs if '*' in e][0].split('*')[1].strip()}")

Environnement virtuel : C:\Users\chrab\anaconda3\envs\opc5


## 1.2 Import des modules

* Installation conditionnelle des librairies

In [2]:
import sys
import subprocess
import pkg_resources

def install_package(package):
    """Installe une librairie en mode silencieux si elle n'est pas encore installée"""
    try:
        pkg_resources.get_distribution(package)
    except pkg_resources.DistributionNotFound:
        print(f"Installation {package}... ", end='')
        subprocess.check_call([sys.executable, "-m", "pip", "install", package, "--quiet"])
        print(f"Terminé.")
    else:
        print(f"{package} est déjà installé.")

In [3]:
# Installation des librairies
install_package('pandas')
install_package('scikit-learn')

pandas est déjà installé.
scikit-learn est déjà installé.


* Import des modules

In [4]:
import pandas as pd
from datetime import datetime, timedelta
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score

## 1.3 Fonctions

In [5]:
def get_RFM_features(df, end_date):
    """Renvoit un dataframe contenant une ligne par client ayant commandé avant 'end_date' ainsi que les features :
    - 'Récence'   : nombre de jours depuis la dernière commande
    - 'Fréquence' : 0 si le client a passé une seule commande, 1 s'il en a passé plusieurs
    - 'Montant'   : total des montants des commandes du client
    """
    # Dernière date de commande
    last_purchase_date = df['order_purchase_timestamp'].max()
    
    # Filtre sur la date de fin
    mask = df['order_purchase_timestamp'] < end_date
    
    # Préparation du dataframe
    df_data = df.loc[mask].groupby('customer_unique_id').agg(
        Récence = ('order_purchase_timestamp', 'max'),
        Fréquence = ('customer_id', 'count'),
        Montant = ('total_price', 'sum')
    ).reset_index()
    
    # Récence
    df_data['Récence'] = (last_purchase_date - df_data['Récence']).dt.days

    # Fréquence
    mask = df_data['Fréquence'] > 1
    df_data['Fréquence'] = 0
    df_data.loc[mask, 'Fréquence'] = 1

    return df_data

In [6]:
def add_transformed_RFM_features(df, recence_t=MinMaxScaler(), montant_t=MinMaxScaler()):
    """Ajoute les features RFM transformées df et renvoit les transformers utilisés"""
    # Récence
    recence_transformer = recence_t.fit(df[['Récence']])
    df['Récence_minmax'] = recence_transformer.transform(df[['Récence']])

    # Fréquence (valeurs déjà comprise entre 0 et 1)
    df['Fréquence_minmax'] = df['Fréquence']

    # Classement par intervalles de montants
    bins = [0, 25, 50, 100, 150, 200, 250, 500, 1000, float('inf')]
    labels = range(len(bins) - 1)
    df['Montant_class'] = pd.cut(df['Montant'], bins=bins, labels=labels, right=False)
    montant_transformer = montant_t.fit(df[['Montant_class']])
    df['Montant_class_minmax'] = montant_transformer.transform(df[['Montant_class']])

    return df, recence_transformer, montant_transformer

In [7]:
def get_kmeans_labels(X, k, random_state=0):
    """Instancie un modèle KMeans et renvoit les labels
    Arguments :
    X (DataFrame)      : données
    k (int)            : nombre de clusters
    random_state (int) : seed
    Retour :
    (numpy.ndarray) : labels des clusters
    """
    # Initialisation du modèle
    kmeans = KMeans(n_clusters=k, random_state=0)
    # Entraînement du modèle
    kmeans.fit(X)

    return kmeans.labels_

In [8]:
def get_date_x_days_before(date_str, x):
    # Conversion de la chaîne de caractères en date
    date_format = "%Y-%m-%d"
    date_obj = datetime.strptime(date_str, date_format).date()
    
    # Calcul de la date x jours avant
    date_before_x_days = date_obj - timedelta(days=x)
    
    # Conversion en string
    result_date_str = date_before_x_days.strftime(date_format)
    return result_date_str

# 2 Stratégie

Simuler un contrat de maintenance, c’est **déterminer combien de temps le modèle entraîné peut rester performant.**  

* **Comment déterminer cette "performance" ?**  
   * Si on entraîne un modèle à la date D0 et qu’on utilise ce modèle entraîné pour prédire les segmentations à la date D1, il est possible de comparer ces prédictions avec la réalité, en réentraînant le modèle à la date D1.  
   * Pour disposer du clustering réel en D1, il suffit de se positionner en D1, et de simuler une prédiction antérieure en D0, sur les données de D1.  
   * On pourra donc comparer :  
      * `modele_D0.predict(datas_D1).labels_` vs `model_D1.fit(datas_D1).labels_`
      * avec `modele_D0 = KMeans(k=6).fit(datas_D0)`  
   * Attention au fait que les datas_D0 sont en fait des données mises à l’échelle via `MinMaxScaler()`. Il faudra réutiliser la même instance de ce transformer sur les datas_D1 avant le predict pour obtenir une mise à l’échelle cohérente.  

* **Comment mesurer l’efficacité des prédictions ?**  
   * SciKit-learn fournit la méthode `adjusted_rand_score()` qui mesure la divergence entre une liste de labels de clusters prédits et une liste de labels de clusters réels.  
   * Cet indice est proche de 0 pour un clustering aléatoire, négatif pour un clustering particulièrement divergent, et est égal à 1 pour une prédiction parfaite.  
   * Le score de **0.8** sera retenu comme seuil pour un clustering suffisamment efficace pour ne pas nécessiter de réentraîner le modèle.  

# 3 Préparation des données

In [9]:
# Chargements des tables
df_orders = pd.read_csv('df_orders.csv')
df_order_items = pd.read_csv('df_order_items.csv')
df_customers = pd.read_csv('df_customers.csv')

In [10]:
# Fusion des dataframes 'df_customers' et 'df_orders'
df_features = pd.merge(
    df_customers[['customer_unique_id', 'customer_id']],
    df_orders[['customer_id', 'order_id', 'order_purchase_timestamp']],
    on='customer_id',
    how='inner'
)

In [11]:
# Ajout des données permmettant le calcul de la feature 'Montant' par fusion avec le dataframe 'df_order_items'
df_features = pd.merge(
    df_features, #[['customer_unique_id', 'order_id', 'order_purchase_timestamp']],
    df_order_items[['order_id', 'price', 'freight_value']],
    on='order_id',
    how='inner'
)

In [12]:
# Conversion de la date d'achat au format datetime "YYYY-MM-DD 00:00:00"
df_features['order_purchase_timestamp'] = pd.to_datetime(df_features['order_purchase_timestamp']).dt.normalize()

In [13]:
# Suppression des commandes antérieures au 01/01/2017 et postérieures au 31/08/2018
period_mask = (df_features['order_purchase_timestamp'] >= '2017-01-01') & (df_features['order_purchase_timestamp'] <= '2018-08-31')
df_features = df_features.loc[period_mask]

In [14]:
# Ajout du montant total de chaque commande
df_features['total_price'] = df_features['price'] + df_features['freight_value']

In [15]:
# Suppression des variables inutiles
df_features.drop(columns=['price', 'freight_value'], inplace=True)

In [16]:
# Affichage des première et dernières lignes
display(df_features)

,customer_unique_id,customer_id,order_id,order_purchase_timestamp,total_price
0,861eff4711a542e4b93843c6dd7febb0,06b8999e2fba1a1fbc88172c00ba8bc7,00e7ee1b050b8499577073aeb2a297a1,2017-05-16,146.87
1,290c77bc529b7ac935b93aa66c333dc3,18955e83d337fd6b2def6b18a428ac77,29150127e6685892b6eab3eec79f59c7,2018-01-12,335.48
2,060e732b5b29e8181a18229c7b0b2b5e,4e7b3e00288586ebd08712fdd0374a03,b2059ed67ce144a36e2aa97d2c9e9ad2,2018-05-19,157.73
3,259dac757896d24d7702b9acbbff3f3c,b2b6027bc5c5109e529d4dc6358b12c3,951670f92359f4fe4a63112aa7306eba,2018-03-13,173.30
4,345ecd01c38d18a9036ed96c73b8d066,4f2d8ab171c80ec8364f7c12e35b23ad,6b7d50bd145f6fc7f33cebabd7e49d0f,2018-07-29,252.25
...,...,...,...,...,...
112645,1a29b476fee25c95fbafc67c5ac95cf8,17ddf5dd5d51696bb3d7c6291687be6f,6760e20addcf0121e9d58f2f1ff14298,2018-04-07,88.78
112646,d52a67c98be1cf6a5c84435bd38d095d,e7b71a9017aa05c9a7fd292d714858e8,9ec0c8947d973db4f4e8dcf1fbfa8f1b,2018-04-04,129.06
112647,e9f50caf99f032f0bf3c55141f019d99,5e28dfe12db7fb50a4b2f691faecea5e,fed4434add09a6f332ea398efd656a5c,2018-04-08,56.04
112648,73c2643a0a458b49f58cea58833b192e,56b18e2166679b8a959d72dd06da27f9,e31ec91cea1ecf97797787471f98a8c2,2017-11-03,711.07


In [17]:
# Vérification des données manquantes
display(df_features.isna().mean())

customer_unique_id          0.0
customer_id                 0.0
order_id                    0.0
order_purchase_timestamp    0.0
total_price                 0.0
dtype: float64

In [18]:
# Nombre de clients : doit être égal à 95121 (cf Notebook 2)
display(df_features['customer_unique_id'].nunique())

95121

# 4 TROUVER TITRE

## 4.1 Liste des labels de clusters au 31/08/2018

In [19]:
last_date = '2018-08-31'

In [20]:
df_last_features = get_RFM_features(df_features, last_date)
df_last_features

,customer_unique_id,Récence,Fréquence,Montant
0,0000366f3b9a7992bf8c76cfdf3221e2,111,0,141.90
1,0000b849f77a49e4a4ce2b2a4ca5be3f,114,0,27.19
2,0000f46a3911fa3c0805444483337064,537,0,86.22
3,0000f6ccb0745a6a4b88665a16c9f078,321,0,43.62
4,0004aac84e0df4da2b147fca70cf8255,288,0,196.89
...,...,...,...,...
95116,fffcf5a5ff07b0908bd4e2dbc735a684,447,1,2067.42
95117,fffea47cd6d3cc0a88bd621562a9d061,262,0,84.58
95118,ffff371b4d645b6ecea244b27531430a,568,0,112.46
95119,ffff5962728ec6157033ef9805bacc48,119,0,133.69


In [21]:
df_last_transformed_features, _, _ = add_transformed_RFM_features(df_last_features)
df_last_transformed_features

,customer_unique_id,Récence,Fréquence,Montant,Récence_minmax,Fréquence_minmax,Montant_class,Montant_class_minmax
0,0000366f3b9a7992bf8c76cfdf3221e2,111,0,141.90,0.184692,0,3,0.375
1,0000b849f77a49e4a4ce2b2a4ca5be3f,114,0,27.19,0.189684,0,1,0.125
2,0000f46a3911fa3c0805444483337064,537,0,86.22,0.893511,0,2,0.250
3,0000f6ccb0745a6a4b88665a16c9f078,321,0,43.62,0.534110,0,1,0.125
4,0004aac84e0df4da2b147fca70cf8255,288,0,196.89,0.479201,0,4,0.500
...,...,...,...,...,...,...,...,...
95116,fffcf5a5ff07b0908bd4e2dbc735a684,447,1,2067.42,0.743760,1,8,1.000
95117,fffea47cd6d3cc0a88bd621562a9d061,262,0,84.58,0.435940,0,2,0.250
95118,ffff371b4d645b6ecea244b27531430a,568,0,112.46,0.945092,0,3,0.375
95119,ffff5962728ec6157033ef9805bacc48,119,0,133.69,0.198003,0,3,0.375


In [22]:
X = df_last_transformed_features[['Récence_minmax', 'Fréquence_minmax', 'Montant_class_minmax']]

In [23]:
real_labels = get_kmeans_labels(X, 6)
real_labels

array([3, 3, 4, ..., 4, 3, 4])

## 4.2 Calculs des scores ARI

In [40]:
ari_threshold = 0.7
delay_in_days = simul_frequency = 14
ari_results = []
ari_score = 1

In [41]:
while ari_score > ari_threshold:
    # Calcul de la date de simulation
    simul_date = get_date_x_days_before(last_date, delay_in_days)

    # Récupération des features RFM jusqu'à la date de simulation
    df_simul_RFM_features = get_RFM_features(df_features, simul_date)

    # Ajout des features transformées et récupération des transformers
    df_simul_RFM_features, recence_scaler, montant_scaler = add_transformed_RFM_features(df_simul_RFM_features)

    # Fit KMeans sur données de simulation
    X = df_simul_RFM_features[['Récence_minmax', 'Fréquence_minmax', 'Montant_class_minmax']]
    simul_kmeans = KMeans(n_clusters=6, random_state=0).fit(X)

    # Préparation dataset pour prédictions (utilisation des transformers récupérés)
    df_predict_RFM_features, _, _ = add_transformed_RFM_features(df_last_features, recence_scaler, montant_scaler)

    # Récupération des labels prédits
    X = df_predict_RFM_features[['Récence_minmax', 'Fréquence_minmax', 'Montant_class_minmax']]
    predict_labels = simul_kmeans.predict(X)

    # Calcul du score ARI
    ari_score = adjusted_rand_score(real_labels, predict_labels)

    # Mise à jour de la liste des résultats
    ari_results.append((delay_in_days, ari_score))
    print(f"Simulation à J-{delay_in_days} : score ARI = {ari_score}")

    # Mise à jour du délai de simulation
    delay_in_days += simul_frequency   

Simulation à J-14 : score ARI = 0.7119321568636154
Simulation à J-28 : score ARI = 0.7299116947737196
Simulation à J-42 : score ARI = 0.47644476112256573
